In [436]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv


In [437]:
daily_data=pd.read_csv('/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv')
submission=pd.read_csv('/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv')

In [438]:
daily_data.head(4)

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,06:04 AM,07:19 PM
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,06:05 AM,07:18 PM
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,06:05 AM,07:18 PM
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,06:06 AM,07:16 PM


In [439]:
daily_data.duplicated().sum()

0

In [440]:
daily_data.isna().sum()

day_id                         0
city_id                        0
temperature_celsius            0
condition_text              2414
wind_kph                       0
wind_degree                    0
pressure_mb                    0
precip_mm                      0
humidity                       0
cloud                          0
feels_like_celsius             0
visibility_km                  0
uv_index                       0
gust_kph                       0
air_quality_us-epa-index       0
sunrise                        0
sunset                         0
dtype: int64

In [441]:
daily_data.shape

(2893, 17)

In [442]:
# test_data=daily_data[daily_data['condition_text'].isnull()]
# test_data.shape

In [443]:
# train_data=daily_data[daily_data['condition_text'].isnull()==False]
# train_data.shape

In [444]:
train_data=daily_data.copy()

In [445]:
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [446]:
from datetime import datetime

In [447]:
train_data.columns

Index(['day_id', 'city_id', 'temperature_celsius', 'condition_text',
       'wind_kph', 'wind_degree', 'pressure_mb', 'precip_mm', 'humidity',
       'cloud', 'feels_like_celsius', 'visibility_km', 'uv_index', 'gust_kph',
       'air_quality_us-epa-index', 'sunrise', 'sunset'],
      dtype='object')

In [448]:
def convert_to_hours(time_str):
    time_obj = datetime.strptime(time_str, '%I:%M %p')
    return time_obj.hour + time_obj.minute / 60.0

train_data['sunrise'] = train_data['sunrise'].apply(convert_to_hours)
train_data['sunset'] = train_data['sunset'].apply(convert_to_hours)
train_data['daylight_hours'] = train_data['sunset'] - train_data['sunrise']

In [449]:
train_data.drop(columns=['sunrise', 'sunset'], inplace=True)

In [450]:
train_data['temperature_diff'] = train_data['temperature_celsius'] - train_data['feels_like_celsius']

In [451]:
# Convert wind direction and speed to wind components
def wind_components(speed, degree):
    rad = np.deg2rad(degree)
    u = speed * np.cos(rad)
    v = speed * np.sin(rad)
    return u, v

train_data['wind_u'], train_data['wind_v'] = zip(*train_data.apply(lambda row: wind_components(row['wind_kph'], row['wind_degree']), axis=1))

In [452]:
# Binning temperature
def temperature_bin(temp):
    if temp < 0:
        return 'cold'
    elif 0 <= temp < 15:
        return 'mild'
    else:
        return 'hot'

train_data['temperature_bin'] = train_data['temperature_celsius'].apply(temperature_bin)

In [453]:
# Binning air quality index
def air_quality_bin(aqi):
    if aqi <= 2:
        return 'good'
    elif 3 <= aqi < 5:
        return 'moderate'
    else:
        return 'unhealthy'

train_data['air_quality_category'] = train_data['air_quality_us-epa-index'].apply(air_quality_bin)


In [454]:
train_data['air_quality_category'].value_counts()

air_quality_category
good         2495
moderate      313
unhealthy      85
Name: count, dtype: int64

In [456]:
# Calculate humidity-temperature interaction
train_data['humidity_temp_interaction'] = train_data['humidity'] * train_data['temperature_celsius']

# Calculate wind-temperature interaction
train_data['wind_temp_interaction'] = train_data['wind_kph'] * train_data['temperature_celsius']

train_data['temp_precip_interaction'] = train_data['temperature_celsius'] * train_data['precip_mm']
train_data['humidity_wind_interaction'] = train_data['humidity'] * train_data['wind_kph']
train_data['temp_pressure']=train_data['pressure_mb']*train_data['temperature_celsius']
train_data['precipi_humidity']=train_data['precip_mm']*train_data['humidity']
train_data['cloud_visib']=train_data['cloud']*train_data['visibility_km']

In [457]:
train_data.columns

Index(['day_id', 'city_id', 'temperature_celsius', 'condition_text',
       'wind_kph', 'wind_degree', 'pressure_mb', 'precip_mm', 'humidity',
       'cloud', 'feels_like_celsius', 'visibility_km', 'uv_index', 'gust_kph',
       'air_quality_us-epa-index', 'daylight_hours', 'temperature_diff',
       'wind_u', 'wind_v', 'temperature_bin', 'air_quality_category',
       'humidity_temp_interaction', 'wind_temp_interaction',
       'temp_precip_interaction', 'humidity_wind_interaction', 'temp_pressure',
       'precipi_humidity', 'cloud_visib'],
      dtype='object')

In [458]:
numerical_columns = [
    'temperature_celsius', 'wind_kph', 'wind_degree','pressure_mb', 'precip_mm', 'humidity',
    'cloud', 'feels_like_celsius', 'visibility_km', 'uv_index', 'gust_kph','air_quality_us-epa-index',
    'daylight_hours', 'temperature_diff', 'wind_u', 'wind_v',
    'humidity_temp_interaction', 'wind_temp_interaction','temp_precip_interaction', 'humidity_wind_interaction', 'temp_pressure',
       'precipi_humidity', 'cloud_visib'
]

# Compute statistical features for each row
train_data['mean'] = train_data[numerical_columns].mean(axis=1)
train_data['std'] = train_data[numerical_columns].std(axis=1)
train_data['min'] = train_data[numerical_columns].min(axis=1)
train_data['max'] = train_data[numerical_columns].max(axis=1)
train_data['median'] = train_data[numerical_columns].median(axis=1)
train_data['q1'] = train_data[numerical_columns].quantile(0.25, axis=1)
train_data['q3'] = train_data[numerical_columns].quantile(0.75, axis=1)
train_data['iqr'] = train_data['q3'] - train_data['q1']

In [459]:
train_data.columns

Index(['day_id', 'city_id', 'temperature_celsius', 'condition_text',
       'wind_kph', 'wind_degree', 'pressure_mb', 'precip_mm', 'humidity',
       'cloud', 'feels_like_celsius', 'visibility_km', 'uv_index', 'gust_kph',
       'air_quality_us-epa-index', 'daylight_hours', 'temperature_diff',
       'wind_u', 'wind_v', 'temperature_bin', 'air_quality_category',
       'humidity_temp_interaction', 'wind_temp_interaction',
       'temp_precip_interaction', 'humidity_wind_interaction', 'temp_pressure',
       'precipi_humidity', 'cloud_visib', 'mean', 'std', 'min', 'max',
       'median', 'q1', 'q3', 'iqr'],
      dtype='object')

In [460]:
encoded_train_data=pd.get_dummies(train_data,columns=['city_id','temperature_bin','air_quality_category'],dtype='int')

encoded_train_data.head()

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temperature_bin_cold,temperature_bin_hot,temperature_bin_mild,air_quality_category_good,air_quality_category_moderate,air_quality_category_unhealthy
0,D0001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,...,0,0,0,0,0,1,0,1,0,0
1,D0002,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,...,0,0,0,0,0,1,0,1,0,0
2,D0003,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,...,0,0,0,0,0,1,0,1,0,0
3,D0004,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,...,0,0,0,0,0,1,0,1,0,0
4,D0005,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,...,0,0,0,0,0,1,0,1,0,0


In [461]:
from sklearn.preprocessing import StandardScaler

continuous_features = [ 'temperature_celsius',
       'wind_kph', 'wind_degree', 'pressure_mb', 'precip_mm', 'humidity',
       'cloud', 'feels_like_celsius', 'visibility_km', 'uv_index', 'gust_kph',
       'air_quality_us-epa-index', 'daylight_hours', 'temperature_diff',
       'wind_u', 'wind_v', 
       'humidity_temp_interaction', 'wind_temp_interaction', 'mean', 'std',
       'min', 'max', 'median', 'q1', 'q3', 'iqr','temp_precip_interaction', 'humidity_wind_interaction', 'temp_pressure',
       'precipi_humidity', 'cloud_visib']

scaler = StandardScaler()
# Fit and transform the continuous features
encoded_train_data[continuous_features] = scaler.fit_transform(encoded_train_data[continuous_features])

In [462]:
# encoded_train_data['condition_text']=encoded_train_data['condition_text'].astype('category')

In [463]:
encoded_train_data['condition_text'].unique()

array([nan, 'Light Rain with Thunder', 'Clear and Sunny', 'Partly Cloudy',
       'Light Precipitation', 'Cloudy and Overcast', 'Mist or Fog',
       'Rain Showers', 'Moderate to Heavy Rain', 'Thunderstorms'],
      dtype=object)

In [464]:
test_data=encoded_train_data[encoded_train_data['condition_text'].isnull()]
test_data.shape

(2414, 151)

In [465]:
encoded_train_data=encoded_train_data[encoded_train_data['condition_text'].isnull()==False]
encoded_train_data.shape

(479, 151)

In [466]:
encoded_train_data['condition_text']=encoded_train_data['condition_text'].map({
    'Light Rain with Thunder':0,
    'Clear and Sunny':1,
    'Partly Cloudy':2,
    'Light Precipitation':3,
    'Cloudy and Overcast':4,
    'Mist or Fog':5,
    'Rain Showers':6,
    'Moderate to Heavy Rain':7,
    'Thunderstorms':8
})

In [467]:
# final_data=encoded_train_data.groupby(['day_id','city_id']).first().reset_index()

In [468]:
encoded_train_data.describe()

,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,...,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temperature_bin_cold,temperature_bin_hot,temperature_bin_mild,air_quality_category_good,air_quality_category_moderate,air_quality_category_unhealthy
count,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,...,479.000000,479.000000,479.000000,479.000000,479.0,479.000000,479.000000,479.000000,479.000000,479.000000
mean,0.011245,2.843424,0.061079,-0.005378,-0.103287,0.115697,0.204040,0.425943,0.008506,-0.138458,...,0.004175,0.016701,0.002088,0.006263,0.0,0.851775,0.148225,0.835073,0.118998,0.045929
std,0.975363,1.914084,1.009064,0.967479,0.989108,1.185799,0.949490,1.112565,0.969407,1.284459,...,0.064549,0.128284,0.045691,0.078974,0.0,0.355695,0.355695,0.371503,0.324125,0.209550
min,-2.573867,0.000000,-0.923118,-1.504745,-3.037758,-0.176432,-3.660883,-0.984491,-2.515325,-3.834332,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.759642,1.000000,-0.751018,-0.766908,-0.607025,-0.176432,-0.143469,-0.984491,-0.757755,0.102220,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.000000
50%,0.147470,2.000000,-0.179646,-0.067905,-0.120878,-0.176432,0.425525,0.550261,0.121030,0.102220,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.000000
75%,0.752211,4.000000,0.660203,0.689348,0.527317,-0.069697,0.994518,1.317637,0.723786,0.102220,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.000000
max,2.264065,8.000000,6.704357,1.951437,2.471903,20.743646,1.304878,2.085013,3.129179,8.939379,...,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [469]:
X=encoded_train_data.drop(['day_id','condition_text'],axis=1)
y=encoded_train_data['condition_text']

In [470]:
# y.unique()

In [471]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [472]:
X_train.columns

Index(['temperature_celsius', 'wind_kph', 'wind_degree', 'pressure_mb',
       'precip_mm', 'humidity', 'cloud', 'feels_like_celsius', 'visibility_km',
       'uv_index',
       ...
       'city_id_C109', 'city_id_C110', 'city_id_C111', 'city_id_C112',
       'temperature_bin_cold', 'temperature_bin_hot', 'temperature_bin_mild',
       'air_quality_category_good', 'air_quality_category_moderate',
       'air_quality_category_unhealthy'],
      dtype='object', length=149)

In [473]:
X_train.shape

(383, 149)

In [149]:
import optuna

def objective(trial):
    catboost=  CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
#         random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
#         od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
#         od_wait=trial.suggest_int("od_wait", 10, 50),
        loss_function='MultiClass',
        verbose=False
    )
    catboost.fit(X_train,y_train)
    cat_y_pred=catboost.predict(X_test)
    acc=accuracy_score(y_test,cat_y_pred)
    return acc
# def objective(trial):
#     params = {
#         "n_estimators": 1000,
#         "verbosity": 0,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
#         "max_depth": trial.suggest_int("max_depth", 1, 10),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
#     }

#     xgb= XGBClassifier(**params)
#     xgb.fit(X_train, y_train, verbose=False)
#     predictions = xgb.predict(X_test)
#     acc= accuracy_score(y_test,predictions)
#     return acc

In [368]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=50)

In [152]:
print('Best hyperparameters:', study.best_params)
print('Best Accuracy:', study.best_value)

Best hyperparameters: {'iterations': 713, 'learning_rate': 0.05892971168873326, 'depth': 8, 'l2_leaf_reg': 4.40893182854889, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.101622834725874}
Best Accuracy: 0.7291666666666666


In [474]:
cat_params={'iterations': 713, 'learning_rate': 0.05892971168873326, 'depth': 8, 'l2_leaf_reg': 4.40893182854889, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.101622834725874}
cat_model = CatBoostClassifier(
    
     **cat_params,
    loss_function='MultiClass',
    verbose=False)
cat_model.fit(X_train, y_train)

In [475]:
cat_y_pred=cat_model.predict(X_test)
print("Accuracy score : ",accuracy_score(y_test,cat_y_pred))

Accuracy score :  0.6979166666666666


In [476]:
from xgboost import XGBClassifier

In [477]:
xgb_params={'learning_rate': 0.02162226515397641, 'max_depth': 9, 'subsample': 0.941204841701304, 'colsample_bytree': 0.9251226250081745, 'min_child_weight': 5}

In [478]:
#Creating an XGBoost classifier
xgb = XGBClassifier(**xgb_params,eval_metric='mlogloss', )
#Training the model on the training data
xgb.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9251226250081745, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02162226515397641,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [479]:
#Making predictions on the test set
xgb_predictions = xgb.predict(X_test)
#Calculating accuracy
print("Accuracy:",accuracy_score(y_test, xgb_predictions))

Accuracy: 0.6666666666666666


In [480]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [481]:
hmodel = HistGradientBoostingClassifier(max_bins=100, max_iter=300)
hmodel.fit(X_train, y_train)


HistGradientBoostingClassifier(max_bins=100, max_iter=300)

In [482]:
hy_pred = hmodel.predict(X_test)
accuracy = accuracy_score(y_test, hy_pred)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.656


In [483]:
X_train.shape,X_test.shape

((383, 149), (96, 149))

In [484]:
from lightgbm import LGBMClassifier 

lgbmodel = LGBMClassifier() 
  
# Fit the model on the training data 
lgbmodel.fit(X_train, y_train) 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000235 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2309
[LightGBM] [Info] Number of data points in the train set: 383, number of used features: 36
[LightGBM] [Info] Start training from score -3.383086
[LightGBM] [Info] Start training from score -1.373324
[LightGBM] [Info] Start training from score -1.363068
[LightGBM] [Info] Start training from score -1.959051
[LightGBM] [Info] Start training from score -2.210365
[LightGBM] [Info] Start training from score -2.259156
[LightGBM] [Info] Start training from score -3.114822
[LightGBM] [Info] Start training from score -3.645450
[LightGBM] [Info] Start training from score -3.463128
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

LGBMClassifier()

In [485]:
lgbm_pred = lgbmodel.predict(X_test)
accuracy = accuracy_score(y_test, lgbm_pred)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.667


In [486]:
from sklearn.ensemble import RandomForestClassifier

rfclf = RandomForestClassifier(n_estimators = 100)  
rfclf.fit(X_train, y_train)
rfy_pred = rfclf.predict(X_test)
print("ACCURACY OF THE MODEL:", accuracy_score(y_test, rfy_pred))

ACCURACY OF THE MODEL: 0.7083333333333334


In [487]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=300,
                                 learning_rate=0.05,
                                 random_state=42)
gbc.fit(X_train, y_train)

gbpred = gbc.predict(X_test)
acc = accuracy_score(y_test, gbpred)
print("Gradient Boosting Classifier accuracy is : {:.2f}".format(acc))

Gradient Boosting Classifier accuracy is : 0.67


In [488]:
from sklearn.ensemble import AdaBoostClassifier 

adb = AdaBoostClassifier() 
adb.fit(X_train,y_train)
adbpred=adb.predict(X_test)
adbc = accuracy_score(y_test, adbpred)
print("Gradient Boosting Classifier accuracy is : {:.2f}".format(adbc))

Gradient Boosting Classifier accuracy is : 0.54


In [489]:
from sklearn.ensemble import ExtraTreesClassifier

etclf = ExtraTreesClassifier(n_estimators=10,min_samples_split=2, random_state=42)
etclf.fit(X_train,y_train)
etpred=etclf.predict(X_test)
print("Accuracy:",accuracy_score(y_test,etpred))

Accuracy: 0.6354166666666666


In [490]:
from sklearn.tree import DecisionTreeClassifier

dtclf= DecisionTreeClassifier(min_samples_split=4,random_state=42)
dtclf.fit(X_train,y_train)
dtpred=dtclf.predict(X_test)
print("Accuracy:",accuracy_score(y_test,dtpred))

Accuracy: 0.65625


In [491]:
from sklearn.ensemble import VotingClassifier

# Combine the models using majority voting
ensemble = VotingClassifier(estimators=[('ct', cat_model), ('xgb', xgb), ('hgb', hmodel),('lgbm',lgbmodel),('rf',rfclf),('gbc',gbc),('adb',adb),('et',etclf),('dt',dtclf)], voting='soft')

# Fit the ensemble on the training data
ensemble.fit(X_train, y_train)

vpreds=ensemble.predict(X_test)

# Evaluate the performance of the ensemble on the testing data
print(f"Accuracy of the ensemble: {accuracy_score(y_test,vpreds)*100} %")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000225 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2309
[LightGBM] [Info] Number of data points in the train set: 383, number of used features: 36
[LightGBM] [Info] Start training from score -3.383086
[LightGBM] [Info] Start training from score -1.373324
[LightGBM] [Info] Start training from score -1.363068
[LightGBM] [Info] Start training from score -1.959051
[LightGBM] [Info] Start training from score -2.210365
[LightGBM] [Info] Start training from score -2.259156
[LightGBM] [Info] Start training from score -3.114822
[LightGBM] [Info] Start training from score -3.645450
[LightGBM] [Info] Start training from score -3.463128
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

In [492]:
X_test_sub=test_data.drop(['day_id','condition_text'],axis=1)

In [493]:
X_test_sub.shape,X_train.shape

((2414, 149), (383, 149))

In [494]:
# Ensure all columns match the training data
missing_cols = set(X_test_sub.columns) - set(X_train.columns)
missing_cols

set()

In [495]:
encoded_test_data['condition_text']=ensemble.predict(X_test_sub)

In [496]:
encoded_test_data['condition_text']=encoded_test_data['condition_text'].map({
  0:  'Light Rain with Thunder',
   1: 'Clear and Sunny',
    2:'Partly Cloudy',
   3: 'Light Precipitation',
   4: 'Cloudy and Overcast',
    5:'Mist or Fog',
    6:'Rain Showers',
    7:'Moderate to Heavy Rain',
    8:'Thunderstorms'
})

In [497]:
encoded_test_data

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temperature_bin_cold,temperature_bin_hot,temperature_bin_mild,air_quality_category_good,air_quality_category_moderate,air_quality_category_unhealthy
0,D0001,0.750691,Partly Cloudy,-0.567957,0.490914,-1.274335,-0.160286,-1.028924,1.469710,0.480294,...,0,0,0,0,0,1,0,1,0,0
1,D0002,-0.001477,Partly Cloudy,-0.567957,0.105094,-1.274335,-0.160286,-0.051001,1.469710,0.088375,...,0,0,0,0,0,1,0,1,0,0
4,D0005,-0.603211,Clear and Sunny,-0.912862,-0.647256,0.830021,-0.160286,1.029862,-0.943332,-0.639473,...,0,0,0,0,0,1,0,1,0,0
5,D0006,-0.302344,Clear and Sunny,-0.912862,-0.608674,0.830021,-0.160286,0.721044,-0.943332,-0.415520,...,0,0,0,0,0,1,0,1,0,0
7,D0008,-0.151910,Partly Cloudy,1.377302,1.648376,-0.464967,-0.160286,-1.080394,1.469710,-0.303543,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2887,D2888,-0.377561,Clear and Sunny,0.880640,-0.444701,0.182527,-0.160286,-1.852439,-0.943332,-0.471508,...,0,0,0,1,0,1,0,1,0,0
2889,D2890,-0.693471,Clear and Sunny,0.383978,-1.042723,0.506274,-0.160286,-1.286273,-0.943332,-0.706659,...,0,0,0,1,0,1,0,1,0,0
2890,D2891,-0.422691,Clear and Sunny,0.177036,-1.090950,0.344400,-0.160286,-2.058318,-0.943332,-0.505101,...,0,0,0,1,0,1,0,1,0,0
2891,D2892,-0.422691,Clear and Sunny,0.577124,-0.801585,0.506274,-0.160286,-1.492151,-0.878985,-0.505101,...,0,0,0,1,0,1,0,1,0,0


In [498]:
submission_data=submission.copy()

In [499]:
submission_data['condition_text']=submission['condition_text'].combine_first(encoded_test_data['condition_text'])
submission_data

,day_id,condition_text
0,D0001,Partly Cloudy
1,D0002,Partly Cloudy
2,D0003,Light Rain with Thunder
3,D0004,Clear and Sunny
4,D0005,Clear and Sunny
...,...,...
2888,D2889,Clear and Sunny
2889,D2890,Clear and Sunny
2890,D2891,Clear and Sunny
2891,D2892,Clear and Sunny


In [501]:
submission_data.to_csv('submisison_data5.csv',index=False)